<a href="https://colab.research.google.com/github/mwithgod3952/Machine_Learning_Theory_jh/blob/master/%EC%86%8C%EB%85%BC%EB%AC%B8%EB%A6%AC%EB%B7%B0___Tslearn%2C_A_Machine_Learning_Toolkit_for_Time_Series_Data_Romain_Tavenard%2C_Johann_Faouzi%2C_Gilles_Vandewiele%2C_Felix_Divo%2C_Guillau.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### 설치

In [ ]:
!python -m pip install tslearn

#### <font color = 'blue'>***사전학습***</font>

    1. Cross_Validation(교차검증)
        > 교차검증은 train_set을 train set + validation_set으로 분리한 뒤 validation set을 사용해 검증하는 방식이다. 

            만약, train_set을 통해 특정 모델을 활용한 학습과정을 거친 뒤, 일괄적으로 동일한 test_set을 통한 반복적 검증을 한다면, 
            특정 Feature Engineering 등을 통해 일괄적으로 모델의 성능은 올릴 수 있겠으나, 과적합(Overfitting)문제가 발생할 수 있으며, 
            추 후 새로운 데이터가 유입될 시 올바른 결과를 낼 수 없다.
<font color = 'red'>***(Regression의 경우 MSE가, Classification의 경우 ER(Error Rate)현저히 다르게 나타날 수 있다는 것이다.)***</font>

    1-1 교차검증의 장점 

        - 모든 데이터셋을 훈련에 활용할 수 있다.
        - 정확도를 향상시킬 수 있다.
        - 데이터 부족으로 인한 underfitting을 방지할 수 있다.
        - 평가에 사용되는 데이터 편중을 막을 수 있다.
        - 평가 결과에 따라 좀 더 일반화된 모델을 만들 수 있다.

    1-2 교차검증의 단점

        - Iteration 횟수가 많기 때문에, 모델 훈련/평가 시간이 오래 걸린다.

    1-3. 교차검증의 종류
        - K-Fold Cross Validation ( k-겹 교차 검증 )
            * cross-validation-splitter
            * Leave-one-out-cross-validation( LOOCV )
            * shuffle-split-cross-validation
        - Stratified K-Fold Cross Validation ( 계층별 k-겹 교차 검증 )    

    2 Grid-Search
        > 매개변수를 튜닝하여 일반화 성능을 개선
        > 가능한 조합들을 시도

    - 간단한 그리드서치 실습

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

from sklearn.datasets import load_iris
iris = load_iris()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state=0)
print("훈련 세트의 크기: {} 테스트 세트의 크기: {}".format(X_train.shape[0], X_test.shape[0]))

In [ ]:
best_score = 0
gm = [0.001, 0.01, 0.1, 1, 10, 100]

for gamma in gm:
    for C in gm:
        svm = SVC(gamma=gamma, C=C)
        svm.fit(X_train, y_train)
        score = svm.score(X_test, y_test)
        if score > best_score:
            best_score = score
            best_parameters = {'C' : C, 'gamma' : gamma}

print("최고 점수: {:.2f}".format(best_score))
print("최고 매개변수:", best_parameters)

#### <font color = 'blue'>***study_tslearn***</font>

In [ ]:
from tslearn.utils import to_time_series_dataset
from sklearn.model_selection import KFold, GridSearchCV
from tslearn.neighbors import KNeighborsTimeSeriesClassifier

In [ ]:
my_first_time_series = [1, 3, 4, 2]
my_second_time_series = [1, 2, 4, 2, 1]
# array 를 temporal data로 변경 및 formatted_dataset은 2개의 time series를 내포하고 있음.
formatted_dataset = to_time_series_dataset([my_first_time_series, my_second_time_series])

In [ ]:
print(formatted_dataset)

In [ ]:
print(formatted_dataset.shape)

***represented through a three dimenssional numpy array of shape (n,T,d)***

  - n : number of time seriese
  - T : length
  - d : dimentionality

    * 즉 2개의 Temporal data가 5개의 sample 길이 및 1개의 차원으로 구성되어 져있다고 위 데이터셋을 설명할 수 있다.

***Note***

    - scikit-learn’s pipelines and model-selection tools can be used in conjunction with tslearn transformers and estimators

In [ ]:
from sklearn.model_selection import KFold, GridSearchCV
from tslearn.neighbors import KNeighborsTimeSeriesClassifier

***코드 예시***

In [ ]:
knn = KNeighborsTimeSeriesClassifier(metric="dtw")
p_grid = {"n_neighbors" : [1, 5]}
cv = KFold(n_splits=2, shuffle=True, random_state=0)
clf = GridSearchCV(estimator=knn, param_grid=p_grid, cv=cv)
clf.fit(X, y)

In [ ]:
from tslearn.clustering import TimeSeriesKMeans
from tslearn.datasets import CachedDatasets

In [ ]:
X_train = CachedDatasets().load_dataset('Trace')[0]

In [ ]:
euclidean_params = {'metric':'euclidean'}
dba_params = {'metric':'dtw'}
sdtw_params = {'metric':'softdtw', 'metric_params':{'gamma':.01}}

y_preds = []

for params in (euclidean_params, dba_params, sdtw_params):
    km = TimeSeriesKMeans(n_clusters=3, random_state=0, **params)
    y_preds.append(km.fit_predict(X_train))

***Time series clustering with kmeans***

In [ ]:
import pandas as pd
import numpy
import matplotlib.pyplot as plt

from tslearn.clustering import TimeSeriesKMeans
from tslearn.datasets import CachedDatasets
from tslearn.preprocessing import TimeSeriesScalerMeanVariance, TimeSeriesResampler

In [ ]:
#  make df
col_name = ['X_train', 'y_train', 'X_test', 'y_test']
df = pd.DataFrame()
idx = 0
for name in col_name:
    df[name] = CachedDatasets().load_dataset("Trace")[idx].tolist()
    idx  += 1
df.tail()

In [ ]:
seed = 0
numpy.random.seed(seed)
X_train, y_train, X_test, y_test = CachedDatasets().load_dataset("Trace")
X_train = X_train[y_train < 4]  
numpy.random.shuffle(X_train)

X_train = TimeSeriesScalerMeanVariance().fit_transform(X_train[:50])
X_train = TimeSeriesResampler(sz=40).fit_transform(X_train)
sz = X_train.shape[1]

In [ ]:
print("Euclidean k-means")
km = TimeSeriesKMeans(n_clusters=3, verbose=True, random_state=seed)
y_pred = km.fit_predict(X_train)

plt.figure(figsize = (15, 10))
for yi in range(3):
    plt.subplot(3, 3, yi + 1)
    for xx in X_train[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=.2)
    plt.plot(km.cluster_centers_[yi].ravel(), "r-")
    plt.xlim(0, sz)
    plt.ylim(-4, 4)
    plt.text(0.55, 0.85,'Cluster %d' % (yi + 1),
             transform=plt.gca().transAxes)
    if yi == 1:
        plt.title("Euclidean k-means")

In [ ]:
dba_km = TimeSeriesKMeans(n_clusters=3,
                          n_init=2,
                          metric="dtw",
                          verbose=True,
                          max_iter_barycenter=10,
                          random_state=seed)
y_pred = dba_km.fit_predict(X_train)

In [ ]:
plt.figure(figsize = (15, 10))
for yi in range(3):
    plt.subplot(3, 3, 4 + yi)
    for xx in X_train[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=.2)
    plt.plot(dba_km.cluster_centers_[yi].ravel(), "r-")
    plt.xlim(0, sz)
    plt.ylim(-4, 4)
    plt.text(0.55, 0.85,'Cluster %d' % (yi + 1),
             transform=plt.gca().transAxes)
    if yi == 1:
        plt.title("DBA $k$-means")

In [ ]:
# Soft-DTW-k-means
print("Soft-DTW k-means")
sdtw_km = TimeSeriesKMeans(n_clusters=3,
                           metric="softdtw",
                           metric_params={"gamma": .01},
                           verbose=True,
                           random_state=seed)
y_pred = sdtw_km.fit_predict(X_train)

In [ ]:
plt.figure(figsize = (15, 10))
for yi in range(3):
    plt.subplot(3, 3, 7 + yi)
    for xx in X_train[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=.2)
    plt.plot(sdtw_km.cluster_centers_[yi].ravel(), "r-")
    plt.xlim(0, sz)
    plt.ylim(-4, 4)
    plt.text(0.55, 0.85,'Cluster %d' % (yi + 1),
             transform=plt.gca().transAxes)
    if yi == 1:
        plt.title("Soft-DTW $k$-means")